# Extracción de las cifras del total de fallecidos por Covid-19 en Euskadi

Para la obtención de estos datos utilizo el documento 08.csv de la carpeta situacion-epidemiologica. Estos datos, al igual que el resto de utilizados en este proyecto, se pueden obtener en la web https://opendata.euskadi.eus/inicio/

### 1.- Importación de las librerias necesarias para la limpieza de los datos

In [1]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime

### 2.- Limpieza datos

Antes de importar el documento, elimino las dos primeras filas del encabezamiento del csv en un editor. Son un par de líneas con una breve descripción sobre los datos que incluye el documento, pero innecesarias para su limpieza. Además, así Pandas reconoce automáticamente el nombre de las columnas.

In [2]:
fallecidos_totales = pd.read_csv("situacion-epidemiologica/08.csv", sep=";", encoding="latin-1")

Visualización de los datos importados.

In [3]:
fallecidos_totales.head()

,Data / Fecha,Hildakoak / Fallecidos,Unnamed: 2
0,2020/03/01,0,NaN
1,2020/03/02,0,NaN
2,2020/03/03,0,NaN
3,2020/03/04,1,NaN
4,2020/03/05,0,NaN


Tras visualizar los datos importados y para facilitar el análisis de los mismos voy a dar los siguientes pasos:

2.1.- Eliminación de la última columna, que no posee datos válidos

In [4]:
fallecidos_totales.columns

Index(['Data / Fecha', 'Hildakoak / Fallecidos', 'Unnamed: 2'], dtype='object')

In [5]:
fallecidos_totales.drop(["Unnamed: 2"], axis="columns", inplace=True)

2.2.- Cambio de nombre de las columnas para su simplificación

In [6]:
columnas = ["Fecha", "Fallecidos"]

In [7]:
fallecidos_totales.columns = columnas

2.3.- Cambio del formato de las fechas para que Python las reconozca como tal

In [24]:
def convert_date(string):
    
    """Función para el cambio de formato de las fechas"""
    
    elements = string.split("/")
    result = "/".join([str(elements[2]), str(elements[1]), "20"])
    correct_date = datetime.strptime(result, "%d/%m/%y")
    return correct_date

In [25]:
fallecidos_totales["Fecha"] = fallecidos_totales["Fecha"].apply(lambda x: convert_date(x))

### 3.- Obtención datos

Fallecidos totales en lo que va de pandemia.

In [27]:
fallecidos_pandemia = fallecidos_totales["Fallecidos"].sum()

In [28]:
print(fallecidos_pandemia)

2402


Fallecidos totales en la primera ola. Voy a tomar como referencia el fin del estado de alarma que marcó el confinamiento de la primavera de 2020. El estado de alarma terminó el 21 de junio por lo que voy a incluir los datos entre el 1 de marzo y el 30 de junio, es decir, hasta el registro 121 del csv importado.

In [29]:
primera = fallecidos_totales.iloc[:122]

In [30]:
fallecidos_primera = primera["Fallecidos"].sum()

In [31]:
print(fallecidos_primera)

1618


Fallecidos totales en la segunda ola. Datos desde el 1 de julio y hasta la actualidad, es decir, a partir del registro 122 del csv importado.

In [32]:
segunda = fallecidos_totales[122:]

In [33]:
fallecidos_segunda = segunda["Fallecidos"].sum()

In [34]:
print(fallecidos_segunda)

784


Como Osakidetza facilita estos datos de semana en semana, el Bot, además de publicar las cifras relativas al total de la pandemia, la primera y la segunda ola, también dará al usuario el total de los últimos siete días disponibles.

In [35]:
semana = fallecidos_totales[-7:]

In [38]:
fallecidos_semana = semana["Fallecidos"].sum()

In [39]:
print(fallecidos_semana)

91


Obtención de la última fecha de actualización para que el usuario sepa exactamente a que período hacen referencia los datos.

In [40]:
fallecidos_fecha = fallecidos_totales.iloc[-1]["Fecha"]

In [41]:
fallecidos_fecha = str(fallecidos_fecha).split(" ")[0]

In [42]:
print(fallecidos_fecha)

2020-11-15


### 4.- Exportación datos

Exportación de los datos que el Bot va a incluir en los mensajes que intercambiará con el usuario.

In [43]:
datos_fallecidos = open("../mensajes/fallecidos.pickle", "wb")

In [44]:
pickle.dump(fallecidos_pandemia, datos_fallecidos)
pickle.dump(fallecidos_primera, datos_fallecidos)
pickle.dump(fallecidos_segunda, datos_fallecidos)
pickle.dump(fallecidos_semana, datos_fallecidos)
pickle.dump(fallecidos_fecha, datos_fallecidos)

In [45]:
datos_fallecidos.close()

Exportación de un nuevo csv para hacer una visualización sobre la evolución de los fallecidos en la segunda ola

In [46]:
segunda.reset_index(drop=True, inplace=True)

In [47]:
segunda.to_csv("../graficos/fallecidos.csv")